In [1]:

import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2, remove_semantics, get_merge_pcds, merge_unite_gt
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import matplotlib.pyplot as plt 
import shutil
from tqdm import tqdm 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/media/cedric/Datasets2/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 7
old = False 

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([37.5, 37.5, 37.5]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = '../../pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts) == False:
        os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = '../../pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [3]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [4]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )
        #o3d.visualization.draw_geometries([pcd_nonground_minor])


In [5]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

PointCloud with 11580178 points.


In [6]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

"\npcd_new = o3d.geometry.PointCloud()\npts_num = 1000000\npcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])\n\nmap_labelled = color_pcd_by_labels(pcd_new,                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))\n\no3d.visualization.draw_geometries([map_labelled])\n#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)\n"

Now we subsample the poses based on a voxel_size

In [7]:
if os.path.exists(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz') == False : 
	print(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz')
	poses, positions, \
	sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,out_folder=out_folder)



In [8]:
with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_0.npz') as data:
	poses=data['poses']
	positions=data['positions']
	sampled_indices_local = data['sampled_indices_local']
	sampled_indices_global=data['sampled_indices_global']

Now we can split the point cloud into chunks based on a tbd chunk_size

In [9]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels, obbs = chunk_and_downsample_point_clouds(dataset,pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

100%|██████████| 681/681 [00:45<00:00, 14.91it/s]


Downsampled from (217082, 3) to (2653, 3) points (non-ground)
Downsampled from (176888, 3) to (2000, 3) points (ground)
Downsampled from (163420, 3) to (2426, 3) points (non-ground)
Downsampled from (103759, 3) to (1952, 3) points (ground)
Downsampled from (134970, 3) to (1836, 3) points (non-ground)
Downsampled from (102014, 3) to (1796, 3) points (ground)
Downsampled from (152967, 3) to (2074, 3) points (non-ground)
Downsampled from (98229, 3) to (1774, 3) points (ground)
Downsampled from (66909, 3) to (997, 3) points (non-ground)
Downsampled from (119443, 3) to (2041, 3) points (ground)
Downsampled from (172800, 3) to (2711, 3) points (non-ground)
Downsampled from (102693, 3) to (2278, 3) points (ground)
Downsampled from (145103, 3) to (2172, 3) points (non-ground)
Downsampled from (116571, 3) to (1944, 3) points (ground)
Downsampled from (125594, 3) to (1528, 3) points (non-ground)
Downsampled from (121923, 3) to (1796, 3) points (ground)
Downsampled from (167560, 3) to (1814, 3) p

In [10]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None,semantics=False):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])
    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 and semantics == False : 
            pcd_colors[idcs] = background_color
        else :
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
    
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors)
    return pcd_colored

In [11]:
color_dict_normalized = {
    0: [0.0, 0.0, 0.0],
    1: [0.0, 0.0, 1.0],
    10: [0.9607843137254902, 0.5882352941176471, 0.39215686274509803],
    11: [0.9607843137254902, 0.9019607843137255, 0.39215686274509803],
    13: [0.9803921568627451, 0.3137254901960784, 0.39215686274509803],
    15: [0.5882352941176471, 0.23529411764705882, 0.11764705882352941],
    16: [1.0, 0.0, 0.0],
    18: [0.7058823529411765, 0.11764705882352941, 0.3137254901960784],
    20: [1.0, 0.0, 0.0],
    30: [0.11764705882352941, 0.11764705882352941, 1.0],
    31: [0.7843137254901961, 0.1568627450980392, 1.0],
    32: [0.35294117647058826, 0.11764705882352941, 0.5882352941176471],
    40: [1.0, 0.0, 1.0],
    44: [1.0, 0.5882352941176471, 1.0],
    48: [0.29411764705882354, 0.0, 0.29411764705882354],
    49: [0.29411764705882354, 0.0, 0.6862745098039216],
    50: [0.0, 0.7843137254901961, 1.0],
    51: [0.19607843137254902, 0.47058823529411764, 1.0],
    52: [0.0, 0.5882352941176471, 1.0],
    60: [0.6666666666666666, 1.0, 0.5882352941176471],
    70: [0.0, 0.6862745098039216, 0.0],
    71: [0.0, 0.23529411764705882, 0.5294117647058824],
    72: [0.3137254901960784, 0.9411764705882353, 0.5882352941176471],
    80: [0.5882352941176471, 0.9411764705882353, 1.0],
    81: [0.0, 0.0, 1.0],
    99: [1.0, 1.0, 0.19607843137254902],
    252: [0.9607843137254902, 0.5882352941176471, 0.39215686274509803],
    256: [1.0, 0.0, 0.0],
    253: [0.7843137254901961, 0.1568627450980392, 1.0],
    254: [0.11764705882352941, 0.11764705882352941, 1.0],
    255: [0.35294117647058826, 0.11764705882352941, 0.5882352941176471],
    257: [0.9803921568627451, 0.3137254901960784, 0.39215686274509803],
    258: [0.7058823529411765, 0.11764705882352941, 0.3137254901960784],
    259: [1.0, 0.0, 0.0]
}

reverse_color_dict = {tuple(v): k for k, v in color_dict_normalized.items()}


def color_pcd_kitti(pcd, labels):
    unique_labels = np.unique(labels)
    pcd_colors = np.zeros_like(np.asarray(pcd.points))
    for i in unique_labels:
        idcs = np.where(labels == i)
        idcs = idcs[0]
        pcd_colors[idcs] = np.array(color_dict_normalized[int(i)])

    pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
    return pcd


In [12]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.1
proximity_threshold = 1.0
tarl_norm = False
ncuts_threshold = 0.005
        
out_name = 'tarl_dino_spatial' + str(SEQUENCE_NUM) 
# Generate 30 different colors
 

COLORS = generate_random_colors(400)
COLORS = [(col[0]/255.,col[1]/255.,col[2]/255.) for col in COLORS]


out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'


if os.path.exists(out_kitti_instance) == False : 
        os.makedirs(out_kitti_instance)


out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'

if os.path.exists(out_kitti_semantic) == False : 
        os.makedirs(out_kitti_semantic)


limit = -1 ##use this for experiments to run limit chunks numberss


patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
semantics = np.hstack((kitti_labels_orig['seg_nonground'].reshape(-1,),kitti_labels_orig['seg_ground'].reshape(-1,)))

instances = np.hstack((kitti_labels_orig['instance_nonground'].reshape(-1,),kitti_labels_orig['instance_ground'].reshape(-1,)))
print('number of instances',np.unique(instances))
merge_pcd = o3d.geometry.PointCloud()    



print("total sequence number",len(center_ids))
for sequence in tqdm(range(0,len(center_ids))):
                '''
                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,list(indices),pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,list(sampled_indices_global),
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold,obb=obbs[sequence])
                '''
                pcd_ground_chunk = pcd_ground_chunks[sequence]
                pcd_chunk = pcd_nonground_chunks[sequence]

                inliers = get_statistical_inlier_indices(pcd_ground_chunk)
                ground_inliers = get_subpcd(pcd_ground_chunk, inliers)
                mean_hight = np.mean(np.asarray(ground_inliers.points)[:, 2])
                inliers_ground = np.where(
                    np.asarray(ground_inliers.points)[:, 2] < (mean_hight + 0.2)
                )[0]
                pcd_chunk_ground = get_subpcd(ground_inliers, inliers_ground)
                pcd_chunk_ground.paint_uniform_color([0, 0, 0])
                #o3d.visualization.draw_geometries([merged_chunk])
                
                seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                
                file_name = str(center_ids[sequence]).zfill(6) + '.pcd'

                kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['semantic'][sequence].reshape(-1,),
                                        colors=COLORS,gt_labels=semantics,semantics=True
                                        )
                
                kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_semantic_ground = color_pcd_by_labels(pcd_chunk_ground,seg_ground.reshape(-1,),
                                        colors=COLORS,gt_labels=semantics,semantics=True)
                
                #o3d.visualization.draw_geometries([kitti_chunk_instance + kitti_chunk_instance_ground])
                #o3d.visualization.draw_geometries([pcd_chunk + pcd_chunk_ground])
                #semantic_labels = np.hstack(( kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                #np.savez(out_kitti_semantic + file_name.split('.')[0] + '.npz',labels=semantic_labels)
                semantics_non_ground = color_pcd_kitti(copy.deepcopy(pcd_chunk),kitti_labels['nonground']['semantic'][sequence].reshape(-1,))
                semantics_ground = color_pcd_kitti(copy.deepcopy(pcd_chunk_ground),seg_ground.reshape(-1,))
                kitti_semantics = np.hstack((kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                                                
                
                o3d.io.write_point_cloud(out_folder_ncuts + file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                print('write gt',out_kitti_instance + file_name)
                o3d.io.write_point_cloud(out_kitti_instance + file_name, kitti_chunk_instance + kitti_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(out_kitti_semantic + file_name, semantics_non_ground + semantics_ground, write_ascii=False, compressed=False, print_progress=False)
                
                


        
        
        

number of instances [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131]
total sequence number 69


  1%|▏         | 1/69 [00:02<02:23,  2.11s/it]

write gt ../../pcd_preprocessed/output_chunks/out_kitti_instance7/000046.pcd


  3%|▎         | 2/69 [00:04<02:14,  2.00s/it]

write gt ../../pcd_preprocessed/output_chunks/out_kitti_instance7/000061.pcd


...

write gt ../../pcd_preprocessed/output_chunks/out_kitti_instance7/001022.pcd


 99%|█████████▊| 68/69 [02:26<00:02,  2.41s/it]

write gt ../../pcd_preprocessed/output_chunks/out_kitti_instance7/001060.pcd


100%|██████████| 69/69 [02:28<00:00,  2.16s/it]

write gt ../../pcd_preprocessed/output_chunks/out_kitti_instance7/001091.pcd


In [13]:
out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'
point_cloud_semantis = get_merge_pcds(out_kitti_semantic)[:-1]
seg_pcd = merge_unite_gt(point_cloud_semantis)
cols, labels_kitti_cur = np.unique(
                np.asarray(seg_pcd.colors), axis=0, return_inverse=True
            )
print(cols)
updated_labels = np.zeros(np.asarray(seg_pcd.colors).shape[0])
for label in np.unique(labels_kitti_cur):
                idcs = np.where(labels_kitti_cur == label)[0]
                col_cur = np.asarray(seg_pcd.colors)[idcs][0]
                print(col_cur)
                updated_labels[idcs] = reverse_color_dict[tuple(col_cur)]
    
np.savez(        'semantics_7_'+ str(chunk_size[0]) 
                + ".npz",
                labels=updated_labels,
            )

['000046.pcd', '000061.pcd', '000066.pcd', '000073.pcd', '000085.pcd', '000093.pcd', '000097.pcd', '000110.pcd', '000125.pcd', '000127.pcd', '000149.pcd', '000163.pcd', '000164.pcd', '000176.pcd', '000189.pcd', '000193.pcd', '000201.pcd', '000212.pcd', '000220.pcd', '000225.pcd', '000238.pcd', '000249.pcd', '000252.pcd', '000265.pcd', '000278.pcd', '000293.pcd', '000328.pcd', '000336.pcd', '000349.pcd', '000362.pcd', '000368.pcd', '000373.pcd', '000383.pcd', '000391.pcd', '000393.pcd', '000403.pcd', '000413.pcd', '000415.pcd', '000424.pcd', '000436.pcd', '000441.pcd', '000450.pcd', '000469.pcd', '000483.pcd', '000491.pcd', '000506.pcd', '000515.pcd', '000517.pcd', '000528.pcd', '000539.pcd', '000540.pcd', '000551.pcd', '000562.pcd', '000565.pcd', '000572.pcd', '000581.pcd', '000587.pcd', '000591.pcd', '000601.pcd', '000610.pcd', '000611.pcd', '000622.pcd', '000640.pcd', '000650.pcd', '000755.pcd', '000769.pcd', '000774.pcd', '000779.pcd', '000788.pcd', '000794.pcd', '000796.pcd', '0008

In [14]:
out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'
out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'

point_clouds = get_merge_pcds(out_folder_ncuts)[:-1]
#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)[:-1]
#point_clouds_kitti_semantics = get_merge_pcds(out_kitti_semantic)[:-1]
#point_clouds_kitti_semantic = get_merge_pcds(out_kitti_semantic)[:-1]
instance_pcd = merge_unite_gt(point_clouds_kitti_instances)

point_cloud_semantis = get_merge_pcds(out_kitti_semantic)[:-1]
seg_pcd = merge_unite_gt(point_cloud_semantis)


merge = merge_chunks_unite_instances(point_clouds)
o3d.io.write_point_cloud('tarl_spatial_dino_0.005.pcd',merge)_


#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
unique_colors, label_vec = np.unique(np.asarray(instance_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_seg = np.unique(np.asarray(seg_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_ncuts_all = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)

label_vec_new = label_vec + (labels_seg * np.unique(label_vec).shape[0]) 

new_ncuts_labels = remove_semantics(label_vec,labels_ncuts_all)

idcs = np.where(label_vec == 0)
label_vec_new[idcs] = 0 

metrics_ncuts = Metrics(name='ncuts')

metrics_ncuts.update_stats(labels_ncuts_all,new_ncuts_labels,label_vec_new)

'''
# o3d.visualization.draw_geometries([pcd])
points = np.asarray(instance_pcd.points)
labels = np.unique(label_vec)
points_of_instances = []
for label in labels:
    if label != 0:
        cols = np.zeros((points.shape[0],3))
        idcs = np.where(label_vec == label)
        idcs = idcs[0]
        instance_points = points[idcs]
        points_of_instances.append(instance_points)
        #cols[idcs] = np.array([0,1,0])
        cur_pcd = o3d.geometry.PointCloud()
        cur_pcd.points = o3d.utility.Vector3dVector(instance_points)
        #cur_pcd.colors = o3d.utility.Vector3dVector(cols)
        #o3d.visualization.draw_geometries([cur_pcd])
'''

#instance_pcd = color_pcd_by_labels(instance_pcd,label_vec)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance_fixed" + str(SEQUENCE_NUM) + ".pcd", instance_pcd, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + out_name + str(SEQUENCE_NUM) + ".pcd", merge, write_ascii=False, compressed=False, print_progress=False)
#print(merge)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_semantics" + str(SEQUENCE_NUM) +  ".pcd", merge_kitti_semantics, write_ascii=False, compressed=False, print_progress=False)


SyntaxError: invalid syntax (3286976533.py, line 17)

In [ ]:
import open3d as o3d 
pcd = o3d.io.read_point_cloud('/home/cedric/unsup_3d_instances/point-to-pixel-mapping/pcd_preprocessed/merge_part_kitti_semantic7.pcd')
pcd2 = o3d.io.read_point_cloud('/home/cedric/unsup_3d_instances/point-to-pixel-mapping/pcd_preprocessed/merge_part_kitti_instance7.pcd')
print(pcd)
print(pcd2)
o3d.visualization.draw_geometries([pcd])

PointCloud with 19170491 points.
PointCloud with 19170491 points.


In [ ]:
seg_pcd

PointCloud with 19170491 points.

In [ ]:
merge = merge_chunks_unite_instances(point_clouds)


#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
unique_colors, label_vec = np.unique(np.asarray(instance_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_seg = np.unique(np.asarray(seg_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_ncuts_all = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)

label_vec = label_vec + (labels_seg * np.unique(label_vec).shape[0]) 

new_ncuts_labels = remove_semantics(label_vec,labels_ncuts_all)


metrics_ncuts = Metrics(name='ncuts')

metrics_ncuts.update_stats(labels_ncuts_all,new_ncuts_labels,label_vec)